In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
852,853,3467109,922297.0,8/17/2006 8:00:00 AM,2144,QUEEN ST W,SPADINA AVE,NaN,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,164,Wellington Place,77,Waterfront Communities-The Island (77),D52,629477.5491,4.834157e+06
14245,14246,80792374,NaN,1/9/2018 10:00:00 AM,1817,EGLINTON AVE E,ROSEMOUNT DR,W of,Major Arterial,Scarborough,...,NaN,NaN,NaN,125,Ionview,125,Ionview (125),D41,638918.3838,4.843383e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['MANOEUVER', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 19)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVAGE,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158
17436,11/6/2021 8:00:00 AM,125,MIDDLEFIELD RD,FINCH AVE E,Scarborough,43.809817,-79.257663,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,25 to 29,NaN,NaN,"Automobile, Station Wagon",Lost control,129,Agincourt North
16536,7/3/2020 8:00:00 AM,1920,ORMONT DR,WESTON RD,Etobicoke York,43.764804,-79.545314,Clear,Daylight,Dry,Non-Fatal Injury,Angle,25 to 29,Major,NaN,Motorcycle,Disobeyed Traffic Control,21,Humber Summit


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,...,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
6254,2010-09-09 08:00:00,1120,QUEEN ST W,BAY ST,Toronto and East York,43.651645,-79.383090,Clear,Daylight,Dry,...,Major,NaN,Other,NaN,170,Yonge-Bay Corridor,Thursday,11,2010,9
17838,2022-07-19 08:00:00,1245,MORNINGSIDE AVE,MILITARY TRL,Scarborough,43.789206,-79.194324,Clear,Daylight,Dry,...,NaN,NaN,"Automobile, Station Wagon",Failed to Yield Right of Way,135,Morningside,Tuesday,12,2022,7
12251,2016-01-08 10:00:00,1330,BAYVIEW AVE,PARKHURST BLVD,North York,43.708762,-79.376354,Clear,Daylight,Wet,...,NaN,NaN,"Automobile, Station Wagon",Driving Properly,56,Leaside-Bennington,Friday,13,2016,1


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Twilight',
    'Dawn, artificial': 'Twilight',
    'Dawn': 'Twilight',
    'Dusk': 'Twilight',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Twilight      719
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'SnowIce',
    'Packed Snow': 'SnowIce',
    'Slush': 'SnowIce',
    'Ice': 'SnowIce',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry        15389
Wet         3141
SnowIce      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,...,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
8682,2012-09-10,ALBION RD,FINCH AVE W,Etobicoke York,43.741084,-79.589214,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Driving Properly,2,Mount Olive-Silverstone-Jamestown,Monday,9,2012,9,1
1479,2006-12-31,F G GARDINER XY E,GRAND AVE,Etobicoke York,43.625745,-79.488390,Rain,Dark,Wet,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Driving Properly,161,Humber Bay Shores,Sunday,19,2006,12,0
13738,2017-07-08,BAYVIEW AVE,BLOOR W BAYVIEW N RAMP,Toronto and East York,43.680995,-79.368060,Clear,Daylight,Dry,Fatal,...,0.0,Motorcycle,Driving Properly,98,Rosedale-Moore Park,Saturday,20,2017,7,1
2497,2007-09-03,MCNICOLL AVE,SILVER SPRINGS BLVD,Scarborough,43.811245,-79.304190,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Other,116,Steeles,Monday,6,2007,9,1
5943,2010-06-10,QUEEN ST W,CAMERON ST,Toronto and East York,43.648445,-79.397590,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,Motorcycle,Driving Properly,164,Wellington Place,Thursday,16,2010,6,1


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 8:
        return 'LateNight'
    elif 8 <= hour < 14:
        return 'MorningAfternoon'
    elif 14 <= hour < 19:
        return 'Evening'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
def categorize_vehicle(v):
    if v in ["Automobile, Station Wagon", "Passenger Van", "Taxi", "Other"]:
        return "Car"
    elif v in [
        "Pick Up Truck", "Truck – Open", "Truck – Closed (Blazer, etc)",
        "Truck – Dump", "Truck–Tractor", "Delivery Van",
        "Tow Truck", "Truck (other)", "Truck – Car Carrier",
        "Construction Equipment"
    ]:
        return "Truck"
    elif v in [
        "Municipal Transit Bus (TTC)", "Bus (Other) (Go Bus, Gray Coa)",
        "Intercity Bus", "School Bus"
    ]:
        return "Bus"
    elif v in ["Motorcycle", "Bicycle", "Moped", "Rickshaw"]:
        return "Two Wheeler"
    elif v in ["Police Vehicle", "Fire Vehicle", "Ambulance", "Other Emergency Vehicle"]:
        return "Emergency"
    else:
        return "Car"

In [37]:
def categorize_drivact(a):
    if a == "DrivingProperly":
        return "Driver Proper"
    elif a in ["Failed to Yield Right of Way", "Disobeyed Traffic Control", "Wrong Way on One Way Road"]:
        return "Right-of-Way/Traffic Violation"
    elif a in ["Exceeding Speed Limit", "Speed too Fast For Condition", "Speed too Slow"]:
        return "Speed-Related"
    elif a in ["Lost control", "Improper Turn", "Following too Close", "Improper Lane Change", "Improper Passing"]:
        return "Control/Maneuver Error"
    else:
        return "Other/Unknown"


In [38]:
def categorize_impact(impact):
    if impact in ["Pedestrian Collisions", "Cyclist Collisions"]:
        return "Pedestrian/Cyclist"
    elif impact == "Rear End":
        return "Rear End"
    elif impact in ["Sideswipe", "Approaching"]:
        return "Sideswipe/Approaching"
    elif impact in ["Angle", "Turning Movement"]:
        return "T-Bone"
    elif impact in ["SMV Other", "SMV Unattended Vehicle"]:
        return "Slow/Parked"
    else:
        return "Other/Unknown"

In [39]:
def combine_age_groups(age):
    if age in ["0 to 4", "5 to 9"]:
        return "0–9"
    elif age in ["10 to 14", "15 to 19"]:
        return "10–19"
    elif age in ["20 to 24", "25 to 29"]:
        return "20–29"
    elif age in ["30 to 34", "35 to 39"]:
        return "30–39"
    elif age in ["40 to 44", "45 to 49"]:
        return "40–49"
    elif age in ["50 to 54", "55 to 59"]:
        return "50–59"
    elif age in ["60 to 64", "65 to 69"]:
        return "60–69"
    elif age in ["70 to 74", "75 to 79"]:
        return "70–79"
    elif age in ["80 to 84", "85 to 89"]:
        return "80–89"
    elif age in ["90 to 94", "Over 95"]:
        return "90+"
    else:
        return "Unknown"

In [40]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)
df["VEHICLE_TYPE"] = df["VEHTYPE"].apply(categorize_vehicle)
df["DRIVER_ACTION"] = df["DRIVACT"].apply(categorize_drivact)
df["IMPACT_TYPE"] = df["IMPACTYPE"].apply(categorize_impact)
df["AGE_RANGE"] = df["INVAGE"].apply(combine_age_groups)

In [41]:
df = df[df["IMPACT_TYPE"] != "Other/Unknown"]

In [42]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [43]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [44]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [45]:
df = df[df["HOOD_158"] != "NSA"]
df = df[df["NEIGHBOURHOOD_158"] != "NSA"]
df.rename(columns={
    "NEIGHBOURHOOD_158": "NEIGHBOURHOOD",
    "HOOD_158": "NEIGHBOURHOOD_CODE"
}, inplace=True)

In [46]:
df["NEIGHBOURHOOD"].nunique()

158

In [47]:
df.shape

(18598, 29)

In [48]:
df.to_csv('cleaned_ksi_collisions.csv', index=False)